In [ ]:
# code to find names of companies having profit warnings and save it to data.csv
import urllib2
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

data_name = []
data_date = []
data_symbol = []
data_curr = []
data_range = []
data_period=[]

def scrapperData():
    
    a=soup.find_all('div', attrs={"data-th":"Co. Name"})

    for data in a:
        data_name.append(data.a.string)

    a=soup.find_all('div', class_='tdDate')
    
    for data in a:
        data_date.append(data.string)
        
    a=soup.find_all('div', attrs={"data-th":"Symbol"})
    
    for data in a:
        data_symbol.append(data.a.string)
        
    a=soup.find_all('div', attrs={"data-th":"Curr. Est"})

    for data in a:
        data_curr.append(data.string)
    
    a=soup.find_all('div', attrs={"data-th":"New Range"})

    for data in a:
        data_range.append(data.string)
        
    a=soup.find_all('div', attrs={"data-th":"Period"})

    for data in a:
        data_period.append(data.string)
        

str1= "http://www.rttnews.com/Calendar/ProfitWarnings.aspx?PageNum="
str2="1"

link= str1 + str2

while True:
    try:
        page = urllib2.urlopen(link)
        soup = BeautifulSoup(page)
        scrapperData()
        str2=int(str2)
        str2=str2+1
        str2=str(str2)
        link=str1+str2
        
    except NoSuchElementException:
        print "Page Not Found!"
        break
        
df=pd.DataFrame(data_date,columns=['Date'])
df['Symbol']=data_symbol
df['Co.name']=data_name
df['Curr.Est']=data_curr
df['New Range']=data_range
df['Period']=data_period

file_name='data.csv'
df.to_csv(file_name, sep='\t', encoding='utf-8')

In [ ]:
# code to login to the RTTNEWS site
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import urllib2
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import pickle
import numpy as np
from selenium.webdriver.common.keys import Keys

chrome_options = Options()
chrome_options.add_argument("--headless")  # new window will not open, data scarrping will be done headless
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/path/to/chromedriver")

# Now just tell it wherever you want it to go
driver.get("https://seekingalpha.com/account/login")

# enter details if you want to have full access to the transcipts otherwise publicly available data will be scraped
username = driver.find_element_by_id("login_user_email")
password = driver.find_element_by_id("login_user_password")

username.send_keys("userName")
password.send_keys("password")

# driver.find_element_by_value("Sign in").click()
# submit_button.click()
password.submit()
driver.implicitly_wait(5)
driver.implicitly_wait(5)

In [ ]:
# code to scarpe detailed transcript of each company name having profit warning whose name is saved in data.csv
import numpy as np
b=np.loadtxt(r'data.csv',dtype=list,delimiter='\t',skiprows=1,usecols=(0,))
type(b)

a=[]
for i in b:
    if (i not in a):
    a.append(i)

In [ ]:
# scrape full page
for sym in a:
    search = driver.find_element_by_id('hd-auto')
    search.send_keys(sym)
    search.send_keys(Keys.RETURN)

    time.sleep(2)
    elm = driver.find_element_by_link_text("Earnings")
    elm.click()

    time.sleep(2)
    a=driver.find_element_by_link_text('Transcripts')
    a.click()

    time.sleep(2)
    links=[]
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
    links.append(elem.get_attribute("href"))

    # print links

    time.sleep(2)
    search_links=[]
    str1="results-earnings-call-transcript"

    for i in links:
    if(str1 in i):
    search_links.append(i)

    for j in range(0,min(8,len(search_links))):
    driver.get(search_links[j])

    temp=[]
    temp=search_links[j]

    temp = temp.encode('utf-8')
    temp=temp[-40:-33] 

    driver.find_element_by_link_text('Single page view').click()

    time.sleep(2)
    ll=[]

    data2 = ""

    for i in range(25):
        elems = driver.find_elements_by_xpath("//p[@class = 'p p%d']" % i)


    for elem in elems:
        data2 += elem.text
        data2 = data2.encode('utf-8')

    fn=sym+"-"+temp+".txt"
    with open("/path/to/Data_transcript/" + fn, "w+") as fp: 

    fp.write(data2)
    fp.write("\n")
    fp.write("\n")
    fp.write("\n")

    print "%s done: %d" %(sym,j)